In [3]:
#Imports of all necessary Python Packages
import sklearn as skl
import numpy as np
import pandas as pd
from time import time

#Classifiers
from sklearn.svm import SVC
from sklearn.svm import NuSVC

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis 
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.decomposition import PCA

from sklearn.dummy import DummyClassifier

from sklearn.neighbors import KNeighborsClassifier

from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier

from sklearn.linear_model import LogisticRegressionCV

#Generic utils
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV

#Evaluation
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn import cross_validation
from sklearn.cross_validation import train_test_split

In [4]:
# Importing Bag of Words Features
#BoW_feats = pd.read_csv('BofW_SVD_feats.csv', index_col=0)
#BoW_feats

In [5]:
# Importing Manuly generated Features´
man_feats = pd.read_csv('regras.csv', index_col=0)
#man_feats

In [6]:
# Import POS Featues
pos_tags = pd.read_csv('pos_tags_train_data.csv', index_col=0)
#pos_tags

In [9]:
# Now load the Biling Data 
new_DF = pd.read_csv('new_data_samples.csv', index_col = 0)
new_DF_POS = pd.read_csv('pos_tags_biling_data.csv', index_col=0)
new_DF_POS.drop('labels', axis=1, inplace=True)
new_DF_POS.drop('Unnamed: 0.1', axis=1, inplace=True)
new_DF_man_feats = pd.read_csv('Bregras.csv', index_col= 0)
biling_samples = pd.concat([new_DF['labels'], new_DF_POS, new_DF_man_feats], axis=1)
#biling_samples

In [10]:
#Here import the data to classify
dataframe = pos_tags
dataframe = pd.concat([pos_tags,man_feats], axis=1)
# Uncomment to use biling data
#dataframe = pd.concat([dataframe, biling_samples], axis=0, ignore_index = True)
labels = dataframe['labels'].values
data = dataframe.drop('labels', axis=1).values
print data.shape
print labels.shape


(20078, 766)
(20078,)


In [11]:
t0 = time()
# Feature Extraction
pca = PCA(n_components = 700)
data_decomposed = pca.fit_transform(data)
print("done in %0.3fs." % (time() - t0))

done in 11.554s.


In [12]:
#Generate a Few of the most comon Scikit-Learn Classifiers
#SVM
classfSVM = SVC(C=1, kernel='rbf', gamma='auto', tol=1e-3) #53%
#SVM with control of the %(nu) of support vectors 
classfNuSVM = NuSVC(nu=0.5, kernel='rbf', gamma='auto', tol=1e-3)
#LDA
classfLDA = LinearDiscriminantAnalysis()
#QDA
classfQDA = QuadraticDiscriminantAnalysis()
#KNN
#classfKNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)#34%
#DecisionTree
#classfDecisionTree = DecisionTreeClassifier()
#AdaBoost
classfAdaBoost = AdaBoostClassifier(base_estimator=None, learning_rate=1, n_estimators=50)
#RandomForest
#classfRandForest = RandomForestClassifier(n_estimators=30, n_jobs=-1)

#LogisticRegressionCV
classfLogRegressCV = LogisticRegressionCV(cv = 5, n_jobs=-1)

classf_lst = [classfLDA, classfQDA, classfAdaBoost, classfSVM]

In [18]:
# Create a voting Classifier with the best performing ones.
estimators = [('lda', LinearDiscriminantAnalysis()),  ('logReg', LogisticRegressionCV()),
              ('adaB', AdaBoostClassifier(base_estimator=None, learning_rate=1, n_estimators=50))]
classfVC = VotingClassifier(estimators= estimators, voting='hard')

In [19]:
#Test classifier on all data or preferably on a Test set: 
#DON'T trust these results, there is an high chance of overfitting
#Use ONLY to get a feel of the time Cross-Validation will take

classf = classfVC
train_data, test_data, train_labels, test_labels = train_test_split(data_decomposed,
                                                                    labels, test_size=0.20, random_state=42)

t0 = time()
classf.fit(train_data,train_labels)
print "Score of Classifier: " + str(classf.score(test_data,test_labels))
print("done in %0.3fs." % (time() - t0))

Score of Classifier: 0.555278884462
done in 142.871s.


In [13]:
train_data, test_data, train_labels, test_labels = train_test_split(data_decomposed,
                                                                    labels, test_size=0.20, random_state=42)
for classf in classf_lst:
    print classf
    t0 = time()
    classf.fit(train_data,train_labels)
    print "Score of Classifier: " + str(classf.score(test_data,test_labels))
    print("done in %0.3fs." % (time() - t0))

LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
              solver='svd', store_covariance=False, tol=0.0001)
Score of Classifier: 0.553535856574

/usr/local/lib/python2.7/dist-packages/sklearn/discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/usr/local/lib/python2.7/dist-packages/sklearn/discriminant_analysis.py:688: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")



done in 8.482s.
QuadraticDiscriminantAnalysis(priors=None, reg_param=0.0,
               store_covariances=False, tol=0.0001)
Score of Classifier: 0.519173306773
done in 9.096s.
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')
Score of Classifier: 0.428784860558
done in 20.135s.
AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1,
          n_estimators=50, random_state=None)
Score of Classifier: 0.526145418327
done in 84.995s.
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=-1,
            oob_score=False, random_state=None, 

In [ ]:
# Analyze Confusion Matrix
#test_labels = labels
#test_data = data
#train_data = data
#train_labels

#classf = classfSVM
#classf.fit(train_data)
predicted_labels = classf.predict(test_data)
confusion_matrix(test_labels, predicted_labels)

In [20]:
# Select parameters to use in Cross-Validation
classf_cv = classfVC
data_cv = data_decomposed
N_CV = 5

# Cross Validation
t0 = time()
scores = cross_validation.cross_val_score(classf_cv,data_cv,labels, n_jobs=-2, cv = N_CV)
print "Scores: "
for i,score in enumerate(scores):
    print '\t' + str(i) + ':\t' + str(score) 
print("Accuracy: %0.3f (+/- %0.3f)" % (scores.mean(), scores.std() * 2))
print("\nCross val done in %0.3fs." % (time() - t0))

Scores: 
	0:	0.538710480458
	1:	0.556772908367
	2:	0.554420921544
	3:	0.556413449564
	4:	0.541220423412
Accuracy: 0.550 (+/- 0.016)

Cross val done in 554.092s.


In [27]:
estimators = [('lda', LinearDiscriminantAnalysis()),  ('logReg', LogisticRegressionCV()),
              ('adaB', AdaBoostClassifier(base_estimator=None, learning_rate=1, n_estimators=50))]

pipeline = Pipeline([
    ('pca',PCA()),
    ('VC', VotingClassifier(estimators= estimators, voting='hard'))
])
# Fine tune parameters using exaustive GridSearch:

parameters = {
    'pca__n_components': (675,800,)#,
    #'VC__adaB__n_estimators': (40,50,80)
    
    }
    
grid_search = GridSearchCV(pipeline, parameters,  verbose=1)
#pipeline.get_params()

In [26]:
t0 = time()
grid_search.fit(data, labels)
print("done in %0.3fs" % (time() - t0))
print()

print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Fitting 3 folds for each of 9 candidates, totalling 27 fits
done in 3834.057s
()
Best score: 0.547
Best parameters set:
	VC__adaB__n_estimators: 50
	pca__n_components: 675


[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed: 59.8min finished
